In [1]:
import pandas as pd
import numpy as np
import imblearn as imb
#pd.set_option('display.max_columns', None)

In [38]:
train = pd.read_csv('train.csv')

C:\Continuum\Anaconda2\envs\TFENV\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (294,299,300,302,307,313,315,365,367,369) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
test = pd.read_csv('test.csv')

C:\Continuum\Anaconda2\envs\TFENV\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (294,299,300,302,313,315,365,367,369) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
train.shape

(300000, 377)

In [5]:
train.Responders.value_counts()

0    248303
1     51697
Name: Responders, dtype: int64

In [39]:
def missing_values_table(df): 
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum()/len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        return mis_val_table_ren_columns

In [40]:
mis_val_table_ren_columns = missing_values_table(train)

In [41]:
nan_col = list(mis_val_table_ren_columns[mis_val_table_ren_columns['% of Total Values']> 90].index)

In [42]:
print(len(nan_col))

91


In [43]:
train.drop(nan_col, inplace=True, axis=1)

In [12]:
test.drop(nan_col, inplace=True, axis=1)

In [45]:
test.shape

(200000, 112)

In [14]:
#ind=[]
#for ix,row in train.iterrows():
#    if row.isnull().sum() > 80 :
#        #print (row.isnull().sum())
#        ind.append(ix)

In [15]:
#len(ind)

In [16]:
def confusion_mat(train,y_train,col,val): 
    m_y=[]
    m_n=[]
    for i in range(len(val)):
        m_y.append(0)
        m_n.append(0)
    
    for i in train.index.values:
        for j in range(len(val)):
            if( train.get_value(i,col) == val[j]) and (y_train.get_value(i) == 1):
                m_y[j]+=1
            elif( train.get_value(i,col) == val[j]) and (y_train.get_value(i) == 0):
                m_n[j]+=1
    print ('\tchurned','\tnot churned','\t%churned in all churned','\t%churned for this indivdual category','\n')
    for i in range(len(val)):
        print (val[i],'\t',m_y[i],'\t',m_n[i],'\t',round(100 * m_y[i]/sum(m_y),2),'%','\t',round(100 * m_y[i]/(m_y[i]+m_n[i]),2),'%','\n')

In [46]:
obj_cols = list(train.select_dtypes(include = ['O']).columns)

In [47]:
for col in obj_cols:
    if col != 'city':
        print(col , list(train[col].unique()))

HNW_CATEGORY ['2_Preferred', '1_Imperia', '3_Classic']
OCCUP_ALL_NEW ['HOUSEWIFE', 'SALARIED', 'SELF_EMPLOYED', 'INDIVIDUAL', 'RETIRED', 'STUDENT', 'NON_INDIVIDUA', 'MISSING', nan]
FINAL_WORTH_prev1 ['HIGH', 'MEDIUM', 'LOW', nan]
ENGAGEMENT_TAG_prev1 ['LOW', 'MEDIUM', 'HIGH', nan, 'NO']
EFT_SELF_TRANSFER_PrevQ1 ['N', 'Y']
CC_TAG_LIVE [nan, 'Y']
DEMAT_TAG_LIVE [nan, 'Y']
SEC_ACC_TAG_LIVE [nan, 'Y']
INS_TAG_LIVE [nan, 'Y']
FD_TAG_LIVE [nan, 'Y']
Billpay_Active_PrevQ1_N ['A_MISSING', 'B_1']
Billpay_Reg_ason_Prev1_N ['A_MISSING', 'B_1']
Charges_cnt_PrevQ1_N ['A_MISSING', 'D_3', 'C_2', 'B_1']
FRX_PrevQ1_N ['A_MISSING', 'B_1']
RBI_Class_Audit ['SEMI-URBAN', 'METROPOLITAN', 'URBAN', 'RURAL']
gender_bin ['Female', 'Male', 'Missin']


In [49]:
for col in obj_cols:
    #print(col)
    #train[col] = train[col].astype('category').cat.codes
    train[col] = train[col].apply(lambda x: x if x not in ['MISSING','Missing','missing','Missin','missing'] else pd.np.nan)
    #test[col] = test[col].apply(lambda x: x if x not in ['MISSING','Missing','missing','Missin','missing'] else pd.np.nan)

In [50]:
train.groupby(['gender_bin','Responders'])['Responders'].count()

gender_bin  Responders
Female      0              99588
            1              19624
Male        0             143919
            1              30908
Name: Responders, dtype: int64

In [51]:
train.groupby(['OCCUP_ALL_NEW','Responders'])['Responders'].count()

OCCUP_ALL_NEW  Responders
HOUSEWIFE      0             47764
               1              9745
INDIVIDUAL     0             23659
               1              4101
NON_INDIVIDUA  0               387
               1                60
RETIRED        0             18143
               1              2883
SALARIED       0             59279
               1             11290
SELF_EMPLOYED  0             79767
               1             20240
STUDENT        0             18603
               1              3238
Name: Responders, dtype: int64

In [52]:
train.groupby(['gender_bin','OCCUP_ALL_NEW','Responders'])['Responders'].count()

gender_bin  OCCUP_ALL_NEW  Responders
Female      HOUSEWIFE      0             47109
                           1              9596
            INDIVIDUAL     0              9564
                           1              1675
            NON_INDIVIDUA  0                90
                           1                17
            RETIRED        0              3705
                           1               560
            SALARIED       0             16399
                           1              3012
            SELF_EMPLOYED  0             13435
                           1              3268
            STUDENT        0              9084
                           1              1452
Male        HOUSEWIFE      0               655
                           1               149
            INDIVIDUAL     0             13866
                           1              2386
            NON_INDIVIDUA  0               255
                           1                36
            RETIRED   

In [ ]:
train.gender_bin[train['OCCUP_ALL_NEW'] == 'HOUSEWIFE'] = 'Female'
test.gender_bin[test['OCCUP_ALL_NEW'] == 'HOUSEWIFE'] = 'Female'

In [53]:
y_train = train['Responders']
train.drop(['Responders'],axis = 1,inplace=True)

In [22]:
confusion_mat(train,y_train,'EFT_SELF_TRANSFER_PrevQ1',list(train['EFT_SELF_TRANSFER_PrevQ1'].unique()))

	churned 	not churned 	%churned in all churned 	%churned for this indivdual category 

N 	 50631 	 246357 	 97.94 % 	 17.05 % 

Y 	 1066 	 1946 	 2.06 % 	 35.39 % 



In [23]:
confusion_mat(train,y_train,'gender_bin',list(train['gender_bin'].unique()))

	churned 	not churned 	%churned in all churned 	%churned for this indivdual category 

Female 	 19624 	 99588 	 38.83 % 	 16.46 % 

Male 	 30908 	 143919 	 61.17 % 	 17.68 % 



ZeroDivisionError: division by zero

In [ ]:
confusion_mat(train,y_train,'HNW_CATEGORY',list(train['HNW_CATEGORY'].unique()))

In [ ]:
confusion_mat(train,y_train,'OCCUP_ALL_NEW',list(train['OCCUP_ALL_NEW'].unique()))

In [ ]:
confusion_mat(train,y_train,'FINAL_WORTH_prev1',list(train['FINAL_WORTH_prev1'].unique()))

In [ ]:
train.drop(['CC_TAG_LIVE','DEMAT_TAG_LIVE','SEC_ACC_TAG_LIVE','INS_TAG_LIVE','FD_TAG_LIVE'],axis=1,inplace=True)
test.drop(['CC_TAG_LIVE','DEMAT_TAG_LIVE','SEC_ACC_TAG_LIVE','INS_TAG_LIVE','FD_TAG_LIVE'],axis=1,inplace=True)

In [57]:
obj_cols = list(train.select_dtypes(include = ['O']).columns)

In [58]:
for col in obj_cols:
    #print(col)
    train[col] = train[col].astype('category').cat.codes
    test[col] = test[col].astype('category').cat.codes

In [60]:
test.dtypes.unique()

array([dtype('int64'), dtype('int8'), dtype('int16'), dtype('float64')], dtype=object)

In [61]:
cor = train.corr()

In [62]:
i=0
cols = []
for col in cor.columns.values:
    for index in cor[col].index:
        if(cor.get_value(index,col) > 0.80 or cor.get_value(index,col) < -0.80) and (index != col ) and sorted((index,col)) not in cols:
            i+=1
            tpl = (index,col)
            cols.append(sorted(tpl))
            #print ( i,sorted(tpl),'=> ',cor.get_value(index,col))

In [63]:
cols_0= [i[0] for i in cols]
cols_1= [i[1] for i in cols]
#cols_0 = list(set(cols_0))
cols_0.sort()
#cols_1 = list(set(cols_1))
cols_1.sort()
col_rem=[]
for col in cols_0:
    if col in cols_1:
        col_rem.append(col)
col_rem = list(set(col_rem))

cols_0 = list(set(cols_0))
for col in col_rem:
    if col in cols_0:
        cols_0.remove(col)
print (len(cols_0),len(col_rem))

32 93


In [ ]:
train.drop(col_rem, axis=1,inplace=True)
test.drop(col_rem,axis=1,inplace=True)

In [66]:
train.drop(cols_0, axis=1,inplace=True)
test.drop(cols_0,axis=1,inplace=True)

In [68]:
for col in train.columns.values:
    train[col] = train[col].apply(lambda x: x if str(x) != str(pd.np.nan) else -1)
    test[col] = test[col].apply(lambda x: x if str(x) != str(pd.np.nan) else -1)

In [69]:
col_sing_val=[]
for col in train.columns.values:
    if len(train[col].unique()) == 1:
        col_sing_val.append(col)

In [70]:
train.drop(col_sing_val,axis=1,inplace=True)
#test.drop(col_sing_val,axis=1,inplace=True)
train.shape

(300000, 113)

In [78]:
#pd.plotting.scatter_matrix(train)
#plt.show()
train.shape

(300000, 112)

In [ ]:
train.index = train['UCIC_ID']
train.drop(['UCIC_ID'],axis=1,inplace=True)

In [35]:
test.index = test['UCIC_ID']
test.drop(['UCIC_ID'],axis=1,inplace=True)

In [79]:
import shutil
train.to_csv('PrunedDS\\train_pruned.csv')
test.to_csv('PrunedDS\\test_pruned.csv')
y_train.to_csv('PrunedDS\\target.csv')
shutil.make_archive("PrunedDS", "zip", "PrunedDS")

'C:\\Continuum\\Anaconda2\\envs\\TFENV\\ChurnPrediction\\PrunedDS.zip'

In [80]:
from sklearn.decomposition import PCA
from sklearn.linear_model import SGDClassifier, LogisticRegression, LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

In [81]:
from imblearn.over_sampling import RandomOverSampler
from sklearn.datasets import make_classification
from collections import Counter


X, y = make_classification(n_samples=5000, n_features=2, n_informative=2,
                            n_redundant=0, n_repeated=0, n_classes=3,
                            n_clusters_per_class=1,
                            weights=[0.01, 0.05, 0.94],
                            class_sep=0.8, random_state=0)


ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_sample(X, y)
print(sorted(Counter(y_resampled).items()))


[(0, 4674), (1, 4674), (2, 4674)]


In [82]:
x_train = train.as_matrix()
y_train = y_train.as_matrix()
x_test = test.as_matrix()

In [90]:
from imblearn.over_sampling import SMOTE
smote= SMOTE()
X_resampled_smt , y_resampled_smt = smote.fit_sample(x_train,y_train)
print(sorted(Counter(y_resampled).items()))

[(0, 248303), (1, 248303)]


In [84]:
ros = RandomOverSampler(random_state=0)
print(sorted(Counter(y_train).items()))
X_resampled, y_resampled = ros.fit_sample(x_train, y_train)
print(sorted(Counter(y_resampled).items()))

[(0, 248303), (1, 51697)]
[(0, 248303), (1, 248303)]


In [85]:
X_train, X_test, Y_train, Y_test = train_test_split(X_resampled, y_resampled, test_size=0.2)

In [45]:
from sklearn.cross_validation import cross_val_score , LeaveOneOut, KFold
from scipy.stats import sem

C:\Continuum\Anaconda2\envs\TFENV\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [47]:
def cross_val(X_train,y_train,model,K):
    #loo=LeaveOneOut(X_train[:].shape[0])
    cv = KFold(len(y_train),K,shuffle=True,random_state=0)
    scores = cross_val_score(model,X_train,y_train,cv=cv)
    print (scores)
    print ("Mean score: {0:3f} (+/- {1:.3f})".format(np.mean(scores),sem(scores)))

In [109]:
model = LogisticRegression()
#model = SGDClassifier()
#model = LogisticRegressionCV()
#model = RandomForestClassifier( n_jobs=4)
#model = XGBClassifier(n_jobs=2)
#model = MultinomialNB()

In [49]:
cross_val(X_train,y_train,model,5)

[ 0.94183017  0.94303837  0.94096178  0.93952704  0.94111206]
Mean score: 0.941294 (+/- 0.001)


In [110]:
model.fit(X_resampled,y_resampled)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=4,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [111]:
y_pred=model.predict_proba(x_test)

In [106]:
y_pred = pd.DataFrame(y_pred,columns=['RESPONDERS_NO','RESPONDERS'])
y_pred.index = test.index
y_pred.drop(['RESPONDERS_NO'],axis=1 , inplace=True)

In [107]:
y_pred.mean()

RESPONDERS    0.267222
dtype: float64

In [108]:
y_pred.to_csv('submission\\sample_submission.csv')
shutil.make_archive("submission", "zip", "submission")

'C:\\Continuum\\Anaconda2\\envs\\TFENV\\ChurnPrediction\\submission.zip'